### Scripps Research CBB - Code Topics

<h1><center>Parallization</center></h1>
<h2><center>-- Basic GNU Parallel and Python Multiprocessing --</center></h2>

__Shang-Fu Chen (Shaun)  
PhD student @ Torkamani lab  
Scripps Research Tranlational Institute (SRTI)__

1. How can I go home earlier?
1. In what situation should I parallelize my jobs?
1. What's GNU Parallel? 
1. (How to run multiprocessing in Python script?)

# Outline

All new computers have multiple cores. Many bioinformatics tools are serial in nature and will therefore not use the multiple cores.  
However, many bioinformatics tasks (especially within NGS) are extremely parallelizeable:

|Good for| Not suitable for |
|:-|:-|
|<p><li>Run the same program on many files</li><li>Run the same program on every sequence</li></p>| <p><li>Different steps of analysis with inter-dependenvies.</li><li> __Ex.__ QC -> Assembly -> Variant calling</li></p> |

<img src='img/et-home-phone.jpg' style="width: 800px;">

# How can we go home earlier?
- Sequential loops
- Run session in background  (`nohup`, `screen`)
- Submit jobs to clusters (TORQUE: `qsub`, SLURM: `srun`)

<img src='img/meme_3.jpg' style="width: 800px;">

# Serial vs Parallel





|   |   |
| :-|:-: |
| <p> Traditionally, software has been written for __serial__ computation: <li>A problem is broken into a discrete series of instructions</li><li>Instructions are executed sequentially one after another</li><li>Executed on a single processor</li><li>Only one instruction may execute at any moment in time </p> | <img src='img/serialProblem.gif' style="width: 1000px;"> |
| <p>In the simplest sense, __parallel__ computing is the simultaneous use of multiple compute </p><p> resources to solve a computational problem:<li>A problem is broken into discrete parts that can be solved concurrently</li><li>Each part is further broken down to a series of instructions</li><li>Instructions from each part execute simultaneously on different processors</li><li>An overall control/coordination mechanism is employed </p> | <img src='img/parallelProblem.gif' style="width: 1000px;"> |


- Resoure: [Introduction to Parallel Computing](https://computing.llnl.gov/tutorials/parallel_comp/)


# In reality...

<img src='img/meme_2.jpg' style="width: 800px;">

----
- UNIX-based parallel (GNU `parallel`)
- Python-based parallel (`multiprocessing`)

# Normal Parallel vs GNU Parallel

- A: If you have 32 different jobs you want to run on 4 CPUs, a straight forward way to parallelize is to run 8 jobs on each CPU:
- B: GNU Parallel instead spawns a new process when one finishes - keeping the CPUs active and thus saving time:

A. Normal Parallel | B. GNU Parallel
:-:|:-: 
![fig_1](img/fig_1.png) | ![fig_2](img/fig_2.png)


- Resoure: [Tool: Gnu Parallel - Parallelize Serial Command Line Programs Without Changing Them](https://www.biostars.org/p/63816/)

# GNU Parallel: Installation

A personal installation does not require root access. It can be done in 10 seconds by doing this:  
```
(wget -O - pi.dk/3 || curl pi.dk/3/ || fetch -o - http://pi.dk/3) | bash
```  

- For Mac user: 
```ruby
brew install parallel
```
- For Debian/Ubuntu user: 
```ruby
sudo apt-get install -y parallel
```
- For RedHat/CentOS 
```ruby
sudo yum install parallel
```
- For other installation options see http://git.savannah.gnu.org/cgit/parallel.git/tree/README



# GNU Parallel: Work on Garibaldi @ Scripps Research

```
qsub -I -l nodes=1:ppn=16
```

  
##### NOTE#1: Do NOT use GNU parallel in login node!!!
  
##### NOTE#2: Watch out for the limitation of RAM

# GNU Parallel: Example_1 - single command line

In [5]:
%%bash

start=`date +%s`
for i in {01..22}; do 
    sleep 0.${i}
done
end=`date +%s`
echo Total execution time: $((end-start)) sec

Total execution time: 7 sec


### Parallelization 

```ruby
parallel -j [core_number] [command] {1} ::: [list]
```

```ruby
parallel -j [core_number] [command] {1} {2} ::: [list_1] ::: {list_2}
```

In [2]:
%%bash

start=`date +%s`
parallel -j 8 sleep 0.{1} ::: {01..22}
end=`date +%s`
echo Total execution time: $((end-start)) sec

Total execution time: 2 sec


In [3]:
%%bash

parallel -j 8 echo {1} + {2} ::: {1..2} ::: $(seq 8 9)

1 + 8
1 + 9
2 + 8
2 + 9


# GNU Parallel: Example_2 - composed command line

In [13]:
%%bash

start=`date +%s`
for GENDER in F M; do
    for CHR in {1..3}; do
        for AGE in 40; do
            for BP in 100 200; do
                printf "${AGE}:${GENDER}\t"
                echo ${CHR}:${BP}
                sleep 0.5
            done
        done
    done
done
end=`date +%s`
echo Total execution time: $((end-start)) sec

40:F	1:100
40:F	1:200
40:F	2:100
40:F	2:200
40:F	3:100
40:F	3:200
40:M	1:100
40:M	1:200
40:M	2:100
40:M	2:200
40:M	3:100
40:M	3:200
Total execution time: 6 sec


### Parallelization 

```ruby
parallel --header : -j [core_number] "[command_1] {p_1} {p_3} {p_2}" ::: p_1 [list_1] ::: p_2 [list_2] ::: p_3 [list_3]
```

In [14]:
%%bash

start=`date +%s`
parallel --header : -j 8 "printf '{AGE}:{GENDER}\t'; echo {CHR}:{BP}; sleep 0.1" ::: GENDER F M ::: CHR {1..3} ::: AGE 40 ::: BP 100 200
end=`date +%s`
echo Total execution time: $((end-start)) sec

40:F	1:100
40:F	1:200
40:F	2:200
40:F	2:100
40:F	3:200
40:F	3:100
40:M	1:100
40:M	1:200
40:M	2:100
40:M	2:200
40:M	3:100
40:M	3:200
Total execution time: 1 sec


# GNU Parallel: Example_3 - bash script

In [20]:
%%bash

for i in $(seq -w 22 1); do echo "printf 'this step takes 0.${i} sec\n'; sleep 0.${i}"; done > example_3.sh

cat example_3.sh

printf 'this step takes 0.22 sec\n'; sleep 0.22
printf 'this step takes 0.21 sec\n'; sleep 0.21
printf 'this step takes 0.20 sec\n'; sleep 0.20
printf 'this step takes 0.19 sec\n'; sleep 0.19
printf 'this step takes 0.18 sec\n'; sleep 0.18
printf 'this step takes 0.17 sec\n'; sleep 0.17
printf 'this step takes 0.16 sec\n'; sleep 0.16
printf 'this step takes 0.15 sec\n'; sleep 0.15
printf 'this step takes 0.14 sec\n'; sleep 0.14
printf 'this step takes 0.13 sec\n'; sleep 0.13
printf 'this step takes 0.12 sec\n'; sleep 0.12
printf 'this step takes 0.11 sec\n'; sleep 0.11
printf 'this step takes 0.10 sec\n'; sleep 0.10
printf 'this step takes 0.09 sec\n'; sleep 0.09
printf 'this step takes 0.08 sec\n'; sleep 0.08
printf 'this step takes 0.07 sec\n'; sleep 0.07
printf 'this step takes 0.06 sec\n'; sleep 0.06
printf 'this step takes 0.05 sec\n'; sleep 0.05
printf 'this step takes 0.04 sec\n'; sleep 0.04
printf 'this step takes 0.03 sec\n'; sleep 0.03
printf 'this step takes 0.02 sec\n'; sle

In [21]:
%%bash

time bash example_3.sh

this step takes 0.22 sec
this step takes 0.21 sec
this step takes 0.20 sec
this step takes 0.19 sec
this step takes 0.18 sec
this step takes 0.17 sec
this step takes 0.16 sec
this step takes 0.15 sec
this step takes 0.14 sec
this step takes 0.13 sec
this step takes 0.12 sec
this step takes 0.11 sec
this step takes 0.10 sec
this step takes 0.09 sec
this step takes 0.08 sec
this step takes 0.07 sec
this step takes 0.06 sec
this step takes 0.05 sec
this step takes 0.04 sec
this step takes 0.03 sec
this step takes 0.02 sec
this step takes 0.01 sec



real	0m2.658s
user	0m0.023s
sys	0m0.043s


### Parallelization 


```ruby
parallel -j [core_number] < ${bash_script}
```

In [22]:
%%bash

time parallel < example_3.sh

this step takes 0.15 sec
this step takes 0.16 sec
this step takes 0.17 sec
this step takes 0.18 sec
this step takes 0.19 sec
this step takes 0.20 sec
this step takes 0.21 sec
this step takes 0.22 sec
this step takes 0.07 sec
this step takes 0.08 sec
this step takes 0.09 sec
this step takes 0.10 sec
this step takes 0.11 sec
this step takes 0.12 sec
this step takes 0.13 sec
this step takes 0.14 sec
this step takes 0.01 sec
this step takes 0.02 sec
this step takes 0.03 sec
this step takes 0.04 sec
this step takes 0.05 sec
this step takes 0.06 sec



real	0m0.591s
user	0m0.189s
sys	0m0.239s


# GNU Parallel: Example_4 - function

In [12]:
%%bash

FUN() {
    echo "it take ${1}.${2} sec in this step"
    sleep ${1}.${2}
}

start=`date +%s`
for i in {0..1}; do
    for j in {2..5}; do
        FUN ${i} ${j}
    done
done
end=`date +%s`
echo Total execution time: $((end-start)) sec

it take 0.2 sec in this step
it take 0.3 sec in this step
it take 0.4 sec in this step
it take 0.5 sec in this step
it take 1.2 sec in this step
it take 1.3 sec in this step
it take 1.4 sec in this step
it take 1.5 sec in this step
Total execution time: 7 sec


### Parallelization 

```ruby
FUNCTION() {
    # ...do something...
}
export -f FUNCTION

parallel -j [core_number] FUNCTION
```

In [24]:
%%bash

FUN() {
    echo "it take ${1}.${2} sec in this step"
    sleep ${1}.${2}
}
export -f FUN

start=`date +%s`
parallel -j 8 FUN {1} {2} ::: {0..1} ::: {2..5}
end=`date +%s`
echo Total execution time: $((end-start)) sec

it take 0.2 sec in this step
it take 0.3 sec in this step
it take 0.4 sec in this step
it take 0.5 sec in this step
it take 1.2 sec in this step
it take 1.3 sec in this step
it take 1.4 sec in this step
it take 1.5 sec in this step
Total execution time: 2 sec


# GNU Parallel: Example_5 - while loop

In [15]:
%%bash

for i in {1..4} {A..D} foo bar; do echo $i; done > file.txt

while read line; do
    echo ${line}
done < file.txt

1
2
3
4
A
B
C
D
foo
bar


### Parallelization 


```ruby
parallel -j [core_number] [command] {1} :::: [file]
```

In [25]:
%%bash

parallel echo "{1}" :::: file.txt

1
2
3
4
A
B
C
D
foo
bar


# For more information...

[Tool: Gnu Parallel - Parallelize Serial Command Line Programs Without Changing Them](https://www.biostars.org/p/63816/)

# GNU Parallel: Built-in Tutorial
```ruby
man parallel_tutorial
```


#### EXAMPLE: Replace a for-loop

It is often faster to write a command using GNU Parallel than making a for loop:

```ruby
for i in *gz; do 
    zcat ${i} > $(basename ${i} .gz).unpacked
done
```

can be written as:

```ruby
parallel 'zcat {} > {.}.unpacked' ::: *.gz
```
The added benefit is that the zcats are run in parallel - one per CPU core.

#### EXAMPLE: Parallelizing BLAT

This will start a blat process for each processor and distribute foo.fa to these in 1 MB blocks:

```ruby
cat foo.fa | parallel --round-robin --pipe --recstart '>' 'blat -noHead genome.fa stdin >(cat) >&2' >foo.psl
```

#### EXAMPLE: Blast on multiple machines

Assume you have a 1 GB fasta file that you want blast, GNU Parallel can then split the fasta file into 100 KB chunks and run 1 jobs per CPU core:

```ruby
cat 1gb.fasta | parallel --block 100k --recstart '>' --pipe blastp -evalue 0.01 -outfmt 6 -db db.fa -query - > results
```
If you have access to the local machine, server1 and server2, GNU Parallel can distribute the jobs to each of the servers. It will automatically detect how many CPU cores are on each of the servers:

```ruby
cat 1gb.fasta | parallel -S :,server1,server2 --block 100k --recstart '>' --pipe blastp -evalue 0.01 -outfmt 6 -db db.fa -query - > result
```

#### EXAMPLE: Run bigWigToWig for each chromosome

If you have one file per chomosome it is easy to parallelize processing each file. Here we do bigWigToWig for chromosome 1..19 + X Y M. These will run in parallel but only one job per CPU core. The {} will be substituted with arguments following the separator ':::'.

```ruby
parallel bigWigToWig -chrom=chr{} wgEncodeCrgMapabilityAlign36mer_mm9.bigWig mm9_36mer_chr{}.map ::: {1..19} X Y M
```

#### EXAMPLE: Running composed commands

GNU Parallel is not limited to running a single command. It can run a composed command. Here is now you process multiple FASTA files using Biopieces (which uses pipes to communicate):

```ruby
parallel 'read_fasta -i {} | extract_seq -l 5 | write_fasta -o {.}_trim.fna -x' ::: *.fna
```
See also: https://github.com/maasha/biopieces/wiki/HowTo#howto-use-biopieces-with-gnu-parallel

#### EXAMPLE: Running experiments

Experiments often have several parameters where every combination should be tested. Assume we have a program called experiment that takes 3 arguments: --age --sex --chr:

```ruby
experiment --age 18 --sex M --chr 22
```
Now we want to run experiment for every combination of ages 1..80, sex M/F, chr 1..22+XY:

```ruby
parallel experiment --age {1} --sex {2} --chr {3} ::: {1..80} ::: M F ::: {1..22} X Y
```
To save the output in different files you could do:

```ruby
parallel experiment --age {1} --sex {2} --chr {3} '>' output.{1}.{2}.{3} ::: {1..80} ::: M F ::: {1..22} X Y
```
But GNU Parallel can structure the output into directories so you avoid having thousands of output files in a single dir:

```ruby
parallel --results outputdir experiment --age {1} --sex {2} --chr {3} ::: {1..80} ::: M F ::: {1..22} X Y
```
This will create files like outputdir/1/80/2/M/3/X/stdout containing the standard output of the job.

If you have many different parameters it may be handy to name them:

```ruby
parallel --result outputdir --header : experiment --age {AGE} --sex {SEX} --chr {CHR} ::: AGE {1..80} ::: SEX M F ::: CHR {1..22} X Y
```
Then the output files will be named like outputdir/AGE/80/CHR/Y/SEX/F/stdout

If you want the output in a CSV/TSV-file that you can read into R or LibreOffice Calc, simply point `--result` to a file ending in .csv/.tsv:

```ruby
parallel --result output.tsv --header : experiment --age {AGE} --sex {SEX} --chr {CHR} ::: AGE {1..80} ::: SEX M F ::: CHR {1..22} X Y
```
It will deal correctly with newlines in the output, so they will be read as newlines in R or LibreOffice Calc.

If one of your parameters take on many different values, these can be read from a file using `::::`

```ruby
echo AGE > age_file
seq 1 80 >> age_file
parallel --results outputdir --header : experiment --age {AGE} --sex {SEX} --chr {CHR} :::: age_file ::: SEX M F ::: CHR {1..22} X Y
```
If you have many experiments, it can be useful to see some experiments picked at random. Think of it as painting a picture by numbers: You can start from the top corner, or you can paint bits at random. If you paint bits at random, you will often see a pattern earlier, than if you painted in the structured way.

With `--shuf` GNU Parallel will shuffle the experiments and run them all, but in random order:

```ruby
parallel --shuf --results outputdir --header : experiment --age {AGE} --sex {SEX} --chr {CHR} :::: age_file ::: SEX M F ::: CHR {1..22} X Y
```

#### EXAMPLE(advanced): Using GNU Parallel to parallelize you own scripts

Assume you have BASH/Perl/Python script called `launch`. It takes one arguments, `ID`:

```ruby
launch ID
```
Using parallel you can run multiple `ID`s in parallel using:

```ruby
parallel launch ::: ID1 ID2 ...
```
But you would like to hide this complexity from the user, so the user only has to do:

```ruby
launch ID1 ID2 ...
```
You can do that using `--shebang-wrap`. Change the shebang line from:

```ruby
#!/usr/bin/env bash
#!/usr/bin/env perl
#!/usr/bin/env python
```
to:

```ruby
#!/usr/bin/parallel --shebang-wrap bash
#!/usr/bin/parallel --shebang-wrap perl
#!/usr/bin/parallel --shebang-wrap python
```
You further develop your script so it now takes an `ID` and a `DIR`:

```ruby
launch ID DIR
```
You would like it to take multiple `ID`s but only one `DIR`, and run the `ID`s in `parallel`. Again just change the `shebang` line to:

```ruby
#!/usr/bin/parallel --shebang-wrap bash
```
And now you can run:

```ruby
launch ID1 ID2 ID3 ::: DIR
```

#### Learn more

See more examples: http://www.gnu.org/software/parallel/man.html

Watch the intro videos: https://www.youtube.com/playlist?list=PL284C9FF2488BC6D1

Walk through the tutorial once a year - your command line will love you for it: http://www.gnu.org/software/parallel/parallel_tutorial.html

Sign up for the email list to get support: https://lists.gnu.org/mailman/listinfo/parallel

<img src='img/batman_slap.jpg' style="width: 1000px;">

# Python Multiprocessing

|  |  | 
:-:|:-: 
![fig_1](img/Python-Multiprocessing-Module-01.jpg) | ![fig_2](img/Python-Multiprocessing.png)

- Resoure: [Python Multiprocessing Module With Example](https://data-flair.training/blogs/python-multiprocessing/)
- Resoure: [Multiprocessing in Python: Comparative study — Pool and Process class](https://medium.com/datadriveninvestor/python-multiprocessing-pool-vs-process-comparative-analysis-6c03c5b54eec)


# Python Multiprocessing: Installation
```
pip install multiprocess
```

In [16]:
import multiprocessing as mp
mp.cpu_count()

8

# Python Multiprocessing: Work on Jupyterhub @ Scripps Research

#### NOTE:  Do NOT use `mp.cpu_count()` in jupyterhub node!!!
1. Test with smaller hard-coded cpu number
1. Export to executable script
1. Submit a job to allocate independent multi-core computing node (`qsub -l nodes=1:ppn=16`).

# Python Multiprocesing: Example

- Resource: [Process or Thread, that is the question](https://blog.tecladocode.com/process-or-thread-that-is-the-question/?fbclid=IwAR1oNpytAdUSn1nXFcDU9IMNf67NGoKyQ2SPMNLrR4PeJ4Wb7LtiUkmolzM)

In [69]:
import multiprocessing as mp
from multiprocessing.pool import Pool

import numpy as np
import urllib.request
import gzip
import time

In [70]:
my_chromosome = np.arange(1,23)
my_chromosome = np.append(my_chromosome, ['X', 'Y'])

print(f"my chromosome list: {my_chromosome}")

my chromosome list: ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' 'X' 'Y']


In [71]:
def cal_gc(my_chrom):
    '''
    calculate the GC content per chromosome using partial human genome.
    '''
    results = []
    for i in my_chrom:
        # Read the first 20,000,000 bytes of the file inside the .gz archive located at `url`
        url = 'https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr' + str(i) + '.fa.gz'
        with urllib.request.urlopen(url) as response:
            with gzip.GzipFile(fileobj=response) as uncompressed:
                file_header = uncompressed.read(20000000) 
                seq = "".join(str(file_header).split('\\n')[1:])
                seq_upper = seq.upper()
                gc_content = (seq_upper.count('C') + seq_upper.count('G')) / len(seq_upper)
        results.append(gc_content)
        print(f"chr{i}: {gc_content}")
    return results

In [72]:
start_time = time.time()
serial_gc = []
for i in my_chromosome:
    serial_gc.append(cal_gc([i])[0])
print(f"--- {time.time() - start_time} seconds ---")

chr1: 0.48681213671736084
chr2: 0.41979776557732856
chr3: 0.4137723183059592
chr4: 0.44288822444941534
chr5: 0.4164688418749256
chr6: 0.41725867904158126
chr7: 0.41367654028575
chr8: 0.40931532536553367
chr9: 0.3859680814392652
chr10: 0.4171523652939197
chr11: 0.43805579177061743
chr12: 0.41601868599689573
chr13: 0.06970360526234458
chr14: 0.0663896243940816
chr15: 0.05024959916539498
chr16: 0.46789192158768345
chr17: 0.4607494207163482
chr18: 0.4055106942438019
chr19: 0.5146766308452773
chr20: 0.4119810149390259
chr21: 0.26046303524131525
chr22: 0.17052838767843756
chrX: 0.4074226129661713
chrY: 0.3883418259401253
--- 46.44958686828613 seconds ---


### Parallelization 

In [67]:
my_ncores=mp.cpu_count()
print(f"my core number: {my_ncores}")

# np.random.shuffle(my_chromosome)

my_chunks = np.array_split(my_chromosome, my_ncores)
print(f"my chunks: {my_chunks}")

my core number: 8
my chunks: [array(['1', '2', '3'], dtype='<U21'), array(['4', '5', '6'], dtype='<U21'), array(['7', '8', '9'], dtype='<U21'), array(['10', '11', '12'], dtype='<U21'), array(['13', '14', '15'], dtype='<U21'), array(['16', '17', '18'], dtype='<U21'), array(['19', '20', '21'], dtype='<U21'), array(['22', 'X', 'Y'], dtype='<U21')]


In [73]:
start_time = time.time()
# multiprocessing init.
pool = Pool(my_ncores)

# passing fixed/iterable parameters to slave function
mp_results = pool.map(cal_gc, my_chunks)
pool.close()
pool.join()

mp_gc = [item for sublist in mp_results for item in sublist]
print(f"--- {time.time() - start_time} seconds ---")

chr13: 0.06970360526234458
chr22: 0.17052838767843756
chr14: 0.0663896243940816
chr10: 0.4171523652939197
chr4: 0.44288822444941534
chr1: 0.48681213671736084
chr16: 0.46789192158768345
chr15: 0.05024959916539498
chr7: 0.41367654028575
chr19: 0.5146766308452773
chrX: 0.4074226129661713
chr11: 0.43805579177061743
chr17: 0.4607494207163482
chr5: 0.4164688418749256
chr2: 0.41979776557732856
chr8: 0.40931532536553367
chr20: 0.4119810149390259
chrY: 0.3883418259401253
chr12: 0.41601868599689573
chr18: 0.4055106942438019
chr6: 0.41725867904158126
chr3: 0.4137723183059592
chr21: 0.26046303524131525
chr9: 0.3859680814392652
--- 12.079432249069214 seconds ---


# The result will be the same.

In [25]:
serial_gc == mp_gc

True

# Discussion: Speedup Factors vs Granularity


|  |  |
:-:|:-: 
<img src='img/image_2.png' style="width: 1000px;"> | <img src='img/fig_breakdown.png' style="width: 1600px;">




<img src="img/intro_mgr.png" style="width: 1800px;">